In [1]:
import time
import itertools
import time, os, fnmatch, shutil
import pickle
import re
# Python 2 & 3 Compatibility
from __future__ import print_function, division

# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
from seaborn import plt
import matplotlib

matplotlib.rcParams.update({'font.size': 22})

%matplotlib inline

from __future__ import print_function, division
import requests

Read in csv data files:

In [2]:
import pandas as pd
folder = '/Users/torrie/Documents/Metis_Project_3_(McNulty)/'

#df_ip_payments = pd.read_csv('Inpatient_Prospective_Payment_System__IPPS__Provider_Summary_for_the_Top_100_Diagnosis-Related_Groups__DRG__-_FY2011.csv')
df_ip_payments = pd.read_csv(folder + 'Medicare_Provider_Charge_Inpatient_DRGALL_FY2014.csv')
df_op_payments = pd.read_csv(folder + 'Medicare_Provider_Charge_Outpatient_APC32_CY2014.csv')
df_hospitals = pd.read_csv(folder + 'Hospital_General_Information.csv')

Clean column titles:

In [3]:
df_ip_payments.columns = df_ip_payments.columns.str.strip()
df_ip_payments.columns = df_ip_payments.columns.str.replace(' ','_')
df_ip_payments.columns

Index([u'DRG_Definition', u'Provider_Id', u'Provider_Name',
       u'Provider_Street_Address', u'Provider_City', u'Provider_State',
       u'Provider_Zip_Code', u'Hospital_Referral_Region_(HRR)_Description',
       u'Total_Discharges', u'Average_Covered_Charges',
       u'Average_Total_Payments', u'Average_Medicare_Payments'],
      dtype='object')

In [4]:
df_hospitals.columns = df_hospitals.columns.str.strip()
df_hospitals.columns = df_hospitals.columns.str.replace(' ','_')
df_hospitals.columns

Index([u'Provider_ID', u'Hospital_Name', u'Address', u'City', u'State',
       u'ZIP_Code', u'County_Name', u'Phone_Number', u'Hospital_Type',
       u'Hospital_Ownership', u'Emergency_Services',
       u'Meets_criteria_for_meaningful_use_of_EHRs',
       u'Hospital_overall_rating', u'Hospital_overall_rating_footnote',
       u'Mortality_national_comparison',
       u'Mortality_national_comparison_footnote',
       u'Safety_of_care_national_comparison',
       u'Safety_of_care_national_comparison_footnote',
       u'Readmission_national_comparison',
       u'Readmission_national_comparison_footnote',
       u'Patient_experience_national_comparison',
       u'Patient_experience_national_comparison_footnote',
       u'Effectiveness_of_care_national_comparison',
       u'Effectiveness_of_care_national_comparison_footnote',
       u'Timeliness_of_care_national_comparison',
       u'Timeliness_of_care_national_comparison_footnote',
       u'Efficient_use_of_medical_imaging_national_comparis

In [5]:
df_op_payments.columns = df_op_payments.columns.str.strip()
df_op_payments.columns = df_op_payments.columns.str.replace(' ','_')
df_op_payments.columns

Index([u'apc', u'provider_id', u'provider_name', u'Provider_Street_Address',
       u'Provider_City', u'Provider_State', u'Provider_Zip_Code',
       u'Hospital_Referral_Region', u'Outpatient_Services',
       u'Average_Estimated_Submitted_Charges', u'Average_Total_Payments'],
      dtype='object')

Check data types and structure:

In [6]:
df_op_payments.dtypes

apc                                     object
provider_id                              int64
provider_name                           object
Provider_Street_Address                 object
Provider_City                           object
Provider_State                          object
Provider_Zip_Code                        int64
Hospital_Referral_Region                object
Outpatient_Services                      int64
Average_Estimated_Submitted_Charges    float64
Average_Total_Payments                 float64
dtype: object

In [7]:
df_ip_payments.dtypes

DRG_Definition                                 object
Provider_Id                                     int64
Provider_Name                                  object
Provider_Street_Address                        object
Provider_City                                  object
Provider_State                                 object
Provider_Zip_Code                               int64
Hospital_Referral_Region_(HRR)_Description     object
Total_Discharges                                int64
Average_Covered_Charges                       float64
Average_Total_Payments                        float64
Average_Medicare_Payments                     float64
dtype: object

In [8]:
df_hospitals.dtypes

Provider_ID                                                        int64
Hospital_Name                                                     object
Address                                                           object
City                                                              object
State                                                             object
ZIP_Code                                                           int64
County_Name                                                       object
Phone_Number                                                       int64
Hospital_Type                                                     object
Hospital_Ownership                                                object
Emergency_Services                                                  bool
Meets_criteria_for_meaningful_use_of_EHRs                         object
Hospital_overall_rating                                           object
Hospital_overall_rating_footnote                   

In [16]:
df_op_payments.head()

,apc,provider_id,provider_name,Provider_Street_Address,Provider_City,Provider_State,Provider_Zip_Code,Hospital_Referral_Region,Outpatient_Services,Average_Estimated_Submitted_Charges,Average_Total_Payments
0,0012 - Level I Debridement & Destruction,40055,SPARKS REGIONAL MEDICAL CENTER,1001 TOWSON AVENUE,FORT SMITH,AR,72902,AR - Fort Smith,279,208.986595,43.367670
1,0012 - Level I Debridement & Destruction,50017,MERCY GENERAL HOSPITAL,4001 J ST,SACRAMENTO,CA,95819,CA - Sacramento,25,239.600000,77.549200
2,0012 - Level I Debridement & Destruction,50320,HIGHLAND HOSPITAL,1411 E 31ST STREET,OAKLAND,CA,94602,CA - Alameda County,58,123.689655,71.485517
3,0012 - Level I Debridement & Destruction,50357,GOLETA VALLEY COTTAGE HOSPITAL,351 S PATTERSON AVE,SANTA BARBARA,CA,93111,CA - Santa Barbara,38,292.386316,52.500000
4,0012 - Level I Debridement & Destruction,30064,BANNER-UNIVERSITY MEDICAL CENTER TUCSON CAMPUS,1501 NORTH CAMPBELL AVENUE,TUCSON,AZ,85724,AZ - Tucson,132,267.625000,53.116894


In [17]:
df_ip_payments.head()

,DRG_Definition,Provider_Id,Provider_Name,Provider_Street_Address,Provider_City,Provider_State,Provider_Zip_Code,Hospital_Referral_Region_(HRR)_Description,Total_Discharges,Average_Covered_Charges,Average_Total_Payments,Average_Medicare_Payments
0,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,10033,UNIVERSITY OF ALABAMA HOSPITAL,619 SOUTH 19TH STREET,BIRMINGHAM,AL,35233,AL - Birmingham,13,1172866.385,251876.3077,244457.9231
1,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,30103,MAYO CLINIC HOSPITAL,5777 EAST MAYO BOULEVARD,PHOENIX,AZ,85054,AZ - Phoenix,20,437531.300,240422.8000,133509.5500
2,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,50108,SUTTER GENERAL HOSPITAL,2801 L STREET,SACRAMENTO,CA,95816,CA - Sacramento,25,815674.120,233197.4800,221681.8000
3,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,50262,RONALD REAGAN U C L A MEDICAL CENTER,757 WESTWOOD PLAZA,LOS ANGELES,CA,90095,CA - Los Angeles,14,1499043.571,415968.7857,366608.9286
4,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,50441,STANFORD HOSPITAL,300 PASTEUR DRIVE,STANFORD,CA,94305,CA - San Mateo County,23,2238699.130,420865.4783,403453.6522


Check the types of data in each column:

In [94]:
df_hospitals[['Hospital_Type','Hospital_Ownership', 'Emergency_Services',\
                  'Meets_criteria_for_meaningful_use_of_EHRs',\
                  'Hospital_overall_rating',\
                  'Mortality_national_comparison',\
                  'Safety_of_care_national_comparison',\
                  'Readmission_national_comparison',\
                  'Patient_experience_national_comparison',\
                  'Effectiveness_of_care_national_comparison',\
                  'Timeliness_of_care_national_comparison',\
                  'Efficient_use_of_medical_imaging_national_comparison']].apply(pd.Series.value_counts)


,Hospital_Type,Hospital_Ownership,Emergency_Services,Meets_criteria_for_meaningful_use_of_EHRs,Hospital_overall_rating,Mortality_national_comparison,Safety_of_care_national_comparison,Readmission_national_comparison,Patient_experience_national_comparison,Effectiveness_of_care_national_comparison,Timeliness_of_care_national_comparison,Efficient_use_of_medical_imaging_national_comparison
False,NaN,NaN,295.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
True,NaN,NaN,4512.0,4363.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,679.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,1774.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,941.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Above the National average,NaN,NaN,NaN,NaN,NaN,400.0,787.0,811.0,1217.0,130.0,1096.0,358.0
Acute Care Hospitals,3370.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Below the National average,NaN,NaN,NaN,NaN,NaN,341.0,665.0,869.0,1084.0,245.0,905.0,373.0


In [97]:
df_hospitals[df_hospitals.Safety_of_care_national_comparison == 'Not Available']

,Provider_ID,Hospital_Name,Address,City,State,ZIP_Code,County_Name,Phone_Number,Hospital_Type,Hospital_Ownership,...,Readmission_national_comparison_footnote,Patient_experience_national_comparison,Patient_experience_national_comparison_footnote,Effectiveness_of_care_national_comparison,Effectiveness_of_care_national_comparison_footnote,Timeliness_of_care_national_comparison,Timeliness_of_care_national_comparison_footnote,Efficient_use_of_medical_imaging_national_comparison,Efficient_use_of_medical_imaging_national_comparison_footnote,Location
3,10007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,3344933541,Acute Care Hospitals,Voluntary non-profit - Private,...,NaN,Same as the National average,NaN,Below the National average,NaN,Above the National average,NaN,Not Available,Results are not available for this reporting p...,NaN
4,10008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,3343353374,Acute Care Hospitals,Proprietary,...,NaN,Not Available,Results are not available for this reporting p...,Same as the National average,NaN,Above the National average,NaN,Not Available,Results are not available for this reporting p...,NaN
8,10018,CALLAHAN EYE HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,...,Results are not available for this reporting p...,Not Available,Results are not available for this reporting p...,Same as the National average,NaN,Same as the National average,NaN,Not Available,Results are not available for this reporting p...,NaN
11,10022,CHEROKEE MEDICAL CENTER,400 NORTHWOOD DR,CENTRE,AL,35960,CHEROKEE,2569275531,Acute Care Hospitals,Voluntary non-profit - Private,...,NaN,Above the National average,NaN,Same as the National average,NaN,Same as the National average,NaN,Not Available,Results are not available for this reporting p...,NaN
16,10032,WEDOWEE HOSPITAL,209 NORTH MAIN STREET,WEDOWEE,AL,36278,RANDOLPH,2563572111,Acute Care Hospitals,Government - Hospital District or Authority,...,NaN,Not Available,Results are not available for this reporting p...,Same as the National average,NaN,Same as the National average,NaN,Not Available,Results are not available for this reporting p...,NaN
18,10034,COMMUNITY HOSPITAL INC,805 FRIENDSHIP ROAD,TALLASSEE,AL,36078,ELMORE,3342836541,Acute Care Hospitals,Voluntary non-profit - Private,...,NaN,Same as the National average,NaN,Same as the National average,NaN,Same as the National average,NaN,Not Available,Results are not available for this reporting p...,NaN
24,10044,MARION REGIONAL MEDICAL CENTER,1256 MILITARY STREET SOUTH,HAMILTON,AL,35570,MARION,2059216200,Acute Care Hospitals,Voluntary non-profit - Private,...,NaN,Above the National average,NaN,Same as the National average,NaN,Above the National average,NaN,Not Available,Results are not available for this reporting p...,NaN
25,10045,FAYETTE MEDICAL CENTER,1653 TEMPLE AVENUE NORTH,FAYETTE,AL,35555,FAYETTE,2059325966,Acute Care Hospitals,Voluntary non-profit - Other,...,NaN,Above the National average,NaN,Same as the National average,NaN,Above the National average,NaN,Same as the National average,NaN,NaN
27,10047,GEORGIANA MEDICAL CENTER,515 N MIRANDA AVENUE,GEORGIANA,AL,36033,BUTLER,3343762205,Acute Care Hospitals,Proprietary,...,NaN,Same as the National average,NaN,Same as the National average,NaN,Above the National average,NaN,Not Available,Results are not available for this reporting p...,NaN
29,10051,GREENE COUNTY HOSPITAL,509 WILSON AVENUE,EUTAW,AL,35462,GREENE,2053723388,Acute Care Hospitals,Government - Local,...,NaN,Not Available,Results are not available for this reporting p...,Same as the National average,NaN,Above the National average,NaN,Not Available,Results are not available for this reporting p...,NaN


In [110]:
df_hospitals = df_hospitals.replace('Not Available','NaN')

In [115]:
df_hospitals[df_hospitals.Safety_of_care_national_comparison == 'NaN']

,Provider_ID,Hospital_Name,Address,City,State,ZIP_Code,County_Name,Phone_Number,Hospital_Type,Hospital_Ownership,...,Readmission_national_comparison_footnote,Patient_experience_national_comparison,Patient_experience_national_comparison_footnote,Effectiveness_of_care_national_comparison,Effectiveness_of_care_national_comparison_footnote,Timeliness_of_care_national_comparison,Timeliness_of_care_national_comparison_footnote,Efficient_use_of_medical_imaging_national_comparison,Efficient_use_of_medical_imaging_national_comparison_footnote,Location
3,10007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,3344933541,Acute Care Hospitals,Voluntary non-profit - Private,...,NaN,Same as the National average,NaN,Below the National average,NaN,Above the National average,NaN,NaN,Results are not available for this reporting p...,NaN
4,10008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,3343353374,Acute Care Hospitals,Proprietary,...,NaN,NaN,Results are not available for this reporting p...,Same as the National average,NaN,Above the National average,NaN,NaN,Results are not available for this reporting p...,NaN
8,10018,CALLAHAN EYE HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,...,Results are not available for this reporting p...,NaN,Results are not available for this reporting p...,Same as the National average,NaN,Same as the National average,NaN,NaN,Results are not available for this reporting p...,NaN
11,10022,CHEROKEE MEDICAL CENTER,400 NORTHWOOD DR,CENTRE,AL,35960,CHEROKEE,2569275531,Acute Care Hospitals,Voluntary non-profit - Private,...,NaN,Above the National average,NaN,Same as the National average,NaN,Same as the National average,NaN,NaN,Results are not available for this reporting p...,NaN
16,10032,WEDOWEE HOSPITAL,209 NORTH MAIN STREET,WEDOWEE,AL,36278,RANDOLPH,2563572111,Acute Care Hospitals,Government - Hospital District or Authority,...,NaN,NaN,Results are not available for this reporting p...,Same as the National average,NaN,Same as the National average,NaN,NaN,Results are not available for this reporting p...,NaN
18,10034,COMMUNITY HOSPITAL INC,805 FRIENDSHIP ROAD,TALLASSEE,AL,36078,ELMORE,3342836541,Acute Care Hospitals,Voluntary non-profit - Private,...,NaN,Same as the National average,NaN,Same as the National average,NaN,Same as the National average,NaN,NaN,Results are not available for this reporting p...,NaN
24,10044,MARION REGIONAL MEDICAL CENTER,1256 MILITARY STREET SOUTH,HAMILTON,AL,35570,MARION,2059216200,Acute Care Hospitals,Voluntary non-profit - Private,...,NaN,Above the National average,NaN,Same as the National average,NaN,Above the National average,NaN,NaN,Results are not available for this reporting p...,NaN
25,10045,FAYETTE MEDICAL CENTER,1653 TEMPLE AVENUE NORTH,FAYETTE,AL,35555,FAYETTE,2059325966,Acute Care Hospitals,Voluntary non-profit - Other,...,NaN,Above the National average,NaN,Same as the National average,NaN,Above the National average,NaN,Same as the National average,NaN,NaN
27,10047,GEORGIANA MEDICAL CENTER,515 N MIRANDA AVENUE,GEORGIANA,AL,36033,BUTLER,3343762205,Acute Care Hospitals,Proprietary,...,NaN,Same as the National average,NaN,Same as the National average,NaN,Above the National average,NaN,NaN,Results are not available for this reporting p...,NaN
29,10051,GREENE COUNTY HOSPITAL,509 WILSON AVENUE,EUTAW,AL,35462,GREENE,2053723388,Acute Care Hospitals,Government - Local,...,NaN,NaN,Results are not available for this reporting p...,Same as the National average,NaN,Above the National average,NaN,NaN,Results are not available for this reporting p...,NaN


In [119]:
df_hospitals.shape

(4807, 29)

Drop the unnecessary columns from the hospital info table

In [122]:
df_hospital_info = df_hospitals.copy()

In [123]:
df_hospital_info = df_hospital_info.drop(['Address','City','State',\
                                          'ZIP_Code','Phone_Number',\
                                          'Hospital_overall_rating_footnote',\
                                          'Mortality_national_comparison_footnote',\
                                          'Safety_of_care_national_comparison_footnote',\
                                          'Readmission_national_comparison_footnote',\
                                          'Patient_experience_national_comparison_footnote',\
                                          'Effectiveness_of_care_national_comparison_footnote',\
                                          'Timeliness_of_care_national_comparison_footnote',\
                                          'Efficient_use_of_medical_imaging_national_comparison_footnote',\
                                          'Location','Efficient_use_of_medical_imaging_national_comparison',\
                                          'Timeliness_of_care_national_comparison','Effectiveness_of_care_national_comparison',\
                                          'Safety_of_care_national_comparison','Hospital_overall_rating', \
                                          'Mortality_national_comparison','Meets_criteria_for_meaningful_use_of_EHRs',\
                                          'Readmission_national_comparison','Patient_experience_national_comparison'],1)




In [124]:
df_hospital_info.dtypes

Provider_ID            int64
Hospital_Name         object
County_Name           object
Hospital_Type         object
Hospital_Ownership    object
Emergency_Services      bool
dtype: object

Rename Coumns so can merge datasets:

In [125]:
df_ip_payments.rename(columns={'Provider_Id': 'Provider_ID'}, inplace=True)


In [126]:
df_ip_payments.columns

Index([u'DRG_Definition', u'Provider_ID', u'Provider_Name',
       u'Provider_Street_Address', u'Provider_City', u'Provider_State',
       u'Provider_Zip_Code', u'Hospital_Referral_Region_(HRR)_Description',
       u'Total_Discharges', u'Average_Covered_Charges',
       u'Average_Total_Payments', u'Average_Medicare_Payments',
       u'Service_type'],
      dtype='object')

In [127]:
df_op_payments.columns

Index([u'DRG_Definition', u'Provider_ID', u'Provider_Name',
       u'Provider_Street_Address', u'Provider_City', u'Provider_State',
       u'Provider_Zip_Code', u'Hospital_Referral_Region_(HRR)_Description',
       u'Total_Discharges', u'Average_Covered_Charges',
       u'Average_Total_Payments', u'Service_type',
       u'Average_Medicare_Payments'],
      dtype='object')

In [128]:
df_op_payments.rename(columns={'apc': 'DRG_Definition','provider_id': 'Provider_ID','provider_name': 'Provider_Name',\
                               'Hospital_Referral_Region': 'Hospital_Referral_Region_(HRR)_Description',\
                               'Outpatient_Services': 'Total_Discharges',\
                               'Average_Estimated_Submitted_Charges': 'Average_Covered_Charges'}, inplace=True)



In [129]:
df_op_payments.columns

Index([u'DRG_Definition', u'Provider_ID', u'Provider_Name',
       u'Provider_Street_Address', u'Provider_City', u'Provider_State',
       u'Provider_Zip_Code', u'Hospital_Referral_Region_(HRR)_Description',
       u'Total_Discharges', u'Average_Covered_Charges',
       u'Average_Total_Payments', u'Service_type',
       u'Average_Medicare_Payments'],
      dtype='object')

Make flag to differentiate inpatient from outpatient data and make a dummy colums for Medicare payments for op dataframe

In [130]:
df_op_payments['Service_type'] = "1"

In [131]:
df_op_payments['Average_Medicare_Payments'] = float(0)

In [132]:
df_op_payments.columns

Index([u'DRG_Definition', u'Provider_ID', u'Provider_Name',
       u'Provider_Street_Address', u'Provider_City', u'Provider_State',
       u'Provider_Zip_Code', u'Hospital_Referral_Region_(HRR)_Description',
       u'Total_Discharges', u'Average_Covered_Charges',
       u'Average_Total_Payments', u'Service_type',
       u'Average_Medicare_Payments'],
      dtype='object')

In [133]:
df_ip_payments['Service_type'] = "0"

Make sure data types for each column match so can merge:

In [134]:
df_ip_payments.dtypes

DRG_Definition                                 object
Provider_ID                                     int64
Provider_Name                                  object
Provider_Street_Address                        object
Provider_City                                  object
Provider_State                                 object
Provider_Zip_Code                               int64
Hospital_Referral_Region_(HRR)_Description     object
Total_Discharges                                int64
Average_Covered_Charges                       float64
Average_Total_Payments                        float64
Average_Medicare_Payments                     float64
Service_type                                   object
dtype: object

In [135]:
df_op_payments.dtypes

DRG_Definition                                 object
Provider_ID                                     int64
Provider_Name                                  object
Provider_Street_Address                        object
Provider_City                                  object
Provider_State                                 object
Provider_Zip_Code                               int64
Hospital_Referral_Region_(HRR)_Description     object
Total_Discharges                                int64
Average_Covered_Charges                       float64
Average_Total_Payments                        float64
Service_type                                   object
Average_Medicare_Payments                     float64
dtype: object

Merge IP and OP data!

In [136]:
df_all_payments = pd.concat([df_ip_payments, df_op_payments])

In [138]:
df_all_payments.head()

,Average_Covered_Charges,Average_Medicare_Payments,Average_Total_Payments,DRG_Definition,Hospital_Referral_Region_(HRR)_Description,Provider_City,Provider_ID,Provider_Name,Provider_State,Provider_Street_Address,Provider_Zip_Code,Service_type,Total_Discharges
0,1172866.385,244457.9231,251876.3077,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,AL - Birmingham,BIRMINGHAM,10033,UNIVERSITY OF ALABAMA HOSPITAL,AL,619 SOUTH 19TH STREET,35233,0,13
1,437531.300,133509.5500,240422.8000,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,AZ - Phoenix,PHOENIX,30103,MAYO CLINIC HOSPITAL,AZ,5777 EAST MAYO BOULEVARD,85054,0,20
2,815674.120,221681.8000,233197.4800,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,CA - Sacramento,SACRAMENTO,50108,SUTTER GENERAL HOSPITAL,CA,2801 L STREET,95816,0,25
3,1499043.571,366608.9286,415968.7857,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,CA - Los Angeles,LOS ANGELES,50262,RONALD REAGAN U C L A MEDICAL CENTER,CA,757 WESTWOOD PLAZA,90095,0,14
4,2238699.130,403453.6522,420865.4783,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,CA - San Mateo County,STANFORD,50441,STANFORD HOSPITAL,CA,300 PASTEUR DRIVE,94305,0,23


Make a copy

In [141]:
df_ip_payments_copy = df_all_payments.copy()

Merge copy with provider dataframe:

In [142]:
df_IPandProvider = pd.merge(df_ip_payments_copy, df_hospital_info, on='Provider_ID', how = 'left')


In [143]:
df_IPandProvider.dtypes

Average_Covered_Charges                       float64
Average_Medicare_Payments                     float64
Average_Total_Payments                        float64
DRG_Definition                                 object
Hospital_Referral_Region_(HRR)_Description     object
Provider_City                                  object
Provider_ID                                     int64
Provider_Name                                  object
Provider_State                                 object
Provider_Street_Address                        object
Provider_Zip_Code                               int64
Service_type                                   object
Total_Discharges                                int64
Hospital_Name                                  object
County_Name                                    object
Hospital_Type                                  object
Hospital_Ownership                             object
Emergency_Services                             object
dtype: object

Then pickle and save to send on to the next part of data processing

In [144]:
with open('IP_OP_Hosp_df.pkl', 'wb') as picklefile:
    pickle.dump(df_IPandProvider, picklefile)
    